In [1]:
from PIL import Image
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import torch
import numpy as np
import pandas as pd
import pickle as pkl
from sklearn.metrics import jaccard_score as IOU
from torchvision import models, transforms, io
from torch.utils.data import Dataset, DataLoader
import torch.nn.utils.prune as prune
import utils
import os
import time
import copy

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
DATASET_PATH = 'ADE20K_2021_17_01/'
index_file = 'index_ade20k.pkl'
with open('{}/{}'.format(DATASET_PATH, index_file), 'rb') as f:
    index_ade20k = pkl.load(f)

objects_mat = index_ade20k['objectPresence']

# Find 150 most common object IDs and non-common object IDs
total_object_counts = np.sum(objects_mat, axis=1)
object_count_ids = np.argsort(total_object_counts)[::-1]
most_common_obj_ids = object_count_ids[:150]
irrelevant_obj_ids = object_count_ids[150:]
# Find image IDs where no irrelevant objects appear
irrelevant_obj_counts = np.sum(objects_mat[irrelevant_obj_ids], axis=0)
good_image_ids = np.argwhere(irrelevant_obj_counts == 0).flatten()
# Only common objects included
common_objects_mat = objects_mat[np.ix_(most_common_obj_ids, good_image_ids)]

# Maps {obj_ids: 0-149}
obj_id_map = {sorted(most_common_obj_ids)[idx]: idx + 1 for idx in range(150)}
obj_id_map[-1] = 0

# Pick out images to train/evaluate on
train_image_ids = []
test_image_ids = []
for i in good_image_ids:
    if 'training' in index_ade20k['folder'][i]:
        train_image_ids.append(i)
    elif 'validation' in index_ade20k['folder'][i]:
        test_image_ids.append(i)
    else:
        raise Exception('Invalid folder name.')

In [3]:
class SegmentationDataset(Dataset):
    def __init__(self, image_ids, root_dir, index_mat, transform=None, target_transform=None):
        """
        Args:
            image_ids (list): list of image IDs from ADE20K
            root_dir (string): Directory with all the images.
            index_mat (array): object array from index_ade20k.pkl
            transform (callable, optional): Optional transform to be applied
                on a sample.
            target_transform (callable, optional): Optional transform to be applied
                on a sample segmentation label.
        """
        self.image_ids = image_ids
        self.root_dir = root_dir
        self.index_ade20k = index_mat
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, idx):
        image_id = self.image_ids[idx]
        img_name = os.path.join(self.root_dir, self.index_ade20k['folder'][image_id], 
                                self.index_ade20k['filename'][image_id])
        img_info = utils.loadAde20K(img_name)
        
        image = io.read_image(img_info['img_name']).float()
        class_mask = Image.fromarray(img_info['class_mask'], mode='I')
        
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(class_mask)
            
        sample = (image, label)

        return sample

In [4]:
input_size = 224
transform = transforms.Compose([
                transforms.Resize(input_size),
                transforms.CenterCrop(input_size),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])

target_transform = transforms.Compose([
                transforms.Resize(input_size, interpolation=0),
                transforms.CenterCrop(input_size),
                transforms.ToTensor()
            ])

/usr/local/lib/python3.7/site-packages/torchvision/transforms/transforms.py:258: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


In [16]:
num_samples = len(test_image_ids)
batch_size = 32
training_data = SegmentationDataset(train_image_ids[:num_samples], './', index_ade20k, transform=transform, target_transform=target_transform)
train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=False)
testing_data = SegmentationDataset(test_image_ids[:num_samples], './', index_ade20k, transform=transform, target_transform=target_transform)
test_dataloader = DataLoader(testing_data, batch_size=batch_size, shuffle=False)

In [12]:
def encode_label(label_arr, obj_id_map):
    """
    Encode labels for evaluating loss
    label_arr (tensor): B x 1 x H x W
    obj_id_map: dictionary mapping label class IDs to new (0-150) range IDs
    """
    convert_label_ids = lambda i: obj_id_map[i-1]
    vect_convert_label_ids = np.vectorize(convert_label_ids)
    encoded_label = vect_convert_label_ids(label_arr.squeeze().cpu().numpy())
    
    return torch.tensor(encoded_label, dtype=torch.long)

def get_parameter_size(model):
    """
    Return model size in terms of parameters
    Each parameter is a float32 - 4 bytes
    """
    num_params = 0
    for p in model.parameters():
        num_params += torch.count_nonzero(p.flatten())
        
    total_bytes = num_params.item() / 4
    kb = total_bytes / 1000
    
    return {"# Params": num_params.item(),
            "Size in KB": kb}

In [13]:
def validate(model, test_dataloader):
    # testing pass
    test_start = time.time()
    running_accuracy = 0
    running_iou = 0
    with torch.no_grad():
        for images, labels in test_dataloader:
            images = images.to(device)
            output = model(images)['out']
            labels = encode_label(labels, obj_id_map).to(device)
            probs = torch.nn.functional.softmax(output, dim=1)
            preds = torch.argmax(probs, dim=1, keepdim=True).squeeze()
            num_correct = torch.sum((preds == labels).to(int)).item()
            iou = IOU(labels.detach().cpu().numpy().reshape(-1), preds.detach().cpu().numpy().reshape(-1), average='weighted')
            print('Testing accuracy: {}'.format(num_correct/(224*224*len(images))))
            print('Testing IOU score: {}'.format(iou))
            running_accuracy += num_correct/(224*224*len(images))
            running_iou += iou

    print("Testing time: {} seconds".format(time.time() - test_start))

    return {"Testing pixel accuracy": running_accuracy / len(test_dataloader),
            "Testing IOU accuracy": running_iou / len(test_dataloader)}

In [17]:
model = models.segmentation.fcn_resnet50(pretrained=False, num_classes=151).to(device=device)
model.load_state_dict(torch.load('../epochs_20_weights.pkl', map_location=torch.device('cpu')))

<All keys matched successfully>

In [18]:
validation_metrics = validate(model, test_dataloader)

Testing accuracy: 0.6961252640704719
Testing IOU score: 0.5567664266454655
Testing accuracy: 0.5719585807955995
Testing IOU score: 0.4300693239409355
Testing accuracy: 0.6724398865991709
Testing IOU score: 0.5434137677671387
Testing accuracy: 0.5736245914381377
Testing IOU score: 0.4454706434037535
Testing accuracy: 0.6390131736288265
Testing IOU score: 0.49749971505331414
Testing accuracy: 0.6801969567123725
Testing IOU score: 0.569534708735073
Testing accuracy: 0.6379326022401148
Testing IOU score: 0.49175185504711705
Testing accuracy: 0.6650147729990433
Testing IOU score: 0.5234921624351464
Testing accuracy: 0.6555281658561862
Testing IOU score: 0.5117365614936726
Testing accuracy: 0.6544855442176871
Testing IOU score: 0.5226432431360427
Testing time: 289.6454048156738 seconds


In [19]:
validation_metrics

{'Testing pixel accuracy': 0.644631953855761,
 'Testing IOU accuracy': 0.5092378407657658}